In [233]:
import numpy as np
import pandas as pd

In [234]:
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['result_set.csv']))
features_train = df[["nouncount","adjectivecount","pronouncount","adverbcount","pos","neg"]]
features_train = features_train.loc[1:4500,:]

class_train = df['class']
class_train = class_train.loc[1:4500]
print(features_train)
training_data = np.array(features_train)
training_data = training_data.astype(float)
training_class = np.array(class_train).T
training_class= training_class.astype(float)
training_class = training_class.reshape(training_class.shape[0],1)

Saving result_set.csv to result_set.csv
      nouncount  adjectivecount  pronouncount  adverbcount       pos       neg
1      0.128026        0.033969      0.057099     0.039817  0.000000  1.655929
2      0.291354        0.032471      0.007835     0.000000  0.000000  0.000000
3      0.255080        0.074860      0.075299     0.000000  0.407414  1.079201
4      0.058196        0.045475      0.022708     0.055304  0.000000  0.839215
5      0.567474        0.000000      0.000000     0.061858  0.000000  0.000000
...         ...             ...           ...          ...       ...       ...
4496   0.322750        0.036312      0.037826     0.035587  0.356427  0.356427
4497   0.229273        0.041890      0.080632     0.022547  1.772495  0.000000
4498   0.191289        0.079100      0.126512     0.000000  0.000000  1.022882
4499   0.827728        0.000000      0.000000     0.000000  0.000000  0.000000
4500   0.746324        0.034356      0.000000     0.000000  0.000000  0.000000

[4500 rows 

In [235]:
np.random.seed(10)
weight1 = np.array(2 * np.random.random((6, 5)) - 1)
weight2 = np.array(2* np.random.random((5, 4)) - 1)
weight3 = np.array(2* np.random.random((4,3)) - 1)
weight4 = np.array(2* np.random.random((3,1))- 1)
print(weight1)

#d/dx(1/1+e^-x)
def sigmoid_derivative(x):
  return x*(1-x)


[[ 0.54264129 -0.9584961   0.26729647  0.49760777 -0.00298598]
 [-0.55040671 -0.60387427  0.52106142 -0.66177833 -0.82332037]
 [ 0.37071964  0.90678669 -0.99210347  0.02438453  0.62524192]
 [ 0.22505213  0.44351063 -0.41624786  0.83554825  0.42915157]
 [ 0.08508874 -0.7156599  -0.25331848  0.34826723 -0.11633365]
 [-0.13197201  0.23553396  0.02627649  0.30079436  0.20207791]]


In [236]:
iteration = 1000
learning_rate = 0.02

while iteration:
  input = np.dot(training_data,weight1) #i/p to 1st hidden layer
  output_layer1 = 1/(1+np.exp(-input)) #activation function - sigmoid (1/1+e^-x)
  input2 = np.dot(output_layer1,weight2)
  output_layer2 = 1/(1+np.exp(-input2))
  input3 = np.dot(output_layer2,weight3)
  output_layer3 = 1/(1+np.exp(-input3))
  input4 = np.dot(output_layer3,weight4)
  output_final = 1/(1+np.exp(-input4))

  #error calculation per layer
  error_final = (training_class - output_final) * sigmoid_derivative(output_final)
  err_layer3 = np.dot(weight4, error_final.T) * (sigmoid_derivative(output_layer3).T)
  err_layer2 = np.dot(weight3, err_layer3) * (sigmoid_derivative(output_layer2).T)
  err_layer1 = np.dot(weight2, err_layer2) * (sigmoid_derivative(output_layer1).T)


  #update weights with gradients at each layer using the computed error in each layer
  weight1 -= (np.dot(training_data.T,err_layer1.T)*learning_rate)
  weight2 -= (np.dot(output_layer1.T,err_layer2.T)*learning_rate)
  weight3 -= (np.dot(output_layer2.T, err_layer3.T)*learning_rate)
  weight4 -= (np.dot(output_layer3.T, error_final)*learning_rate)

  iteration -=1

#final weights after trained
print("weight1")
print(weight1)
print("weight2")
print(weight2)
print("weight3")
print(weight3)
print("weight4")
print(weight4)






weight1
[[ 0.56804153 -0.96537403  0.29907858  0.51094744 -0.0139575 ]
 [-0.54455877 -0.60550804  0.52820514 -0.65874674 -0.82577658]
 [ 0.37332613  0.90604222 -0.98891481  0.02574059  0.6241413 ]
 [ 0.22922779  0.44232258 -0.41113997  0.83770096  0.4273857 ]
 [ 0.11447198 -0.72186008 -0.21507749  0.36288848 -0.1293492 ]
 [-0.07260025  0.21740065  0.09516151  0.32625444  0.17724001]]
weight2
[[ 0.52239078  0.08755155  1.14180872 -0.21922391]
 [-0.8897523  -0.36320467 -0.50711221  0.77233082]
 [-0.99013125  0.29602602  0.409319    0.77637048]
 [-0.70302212  0.76281794  0.07375188  0.67065481]
 [-0.49509655  0.80950174 -0.02986357 -0.53160391]]
weight3
[[ 0.10190126 -0.44144701  0.72980073]
 [ 0.25017463  0.86050251  1.23263852]
 [ 1.70420973  0.73283982  0.985112  ]
 [ 0.35190589  1.16621681  1.13598124]]
weight4
[[4.00224829]
 [4.38899562]
 [6.2361013 ]]


In [237]:
def feed_forward(inputs):
  input = np.dot(inputs,weight1) #i/p to 1st hidden layer
  output_layer1 = 1/(1+np.exp(-input)) #activation function - sigmoid (1/1+e^-x)
  input2 = np.dot(output_layer1,weight2)
  output_layer2 = 1/(1+np.exp(-input2))#2nd hidden layer
  input3 = np.dot(output_layer2,weight3)
  output_layer3 = 1/(1+np.exp(-input3))#3rd hidden layer
  input4 = np.dot(output_layer3,weight4)
  output_final = 1/(1+np.exp(-input4))
  return output_final

In [238]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['result_set.csv']))
features_train = df[["nouncount","adjectivecount","pronouncount","adverbcount","pos","neg"]]
features_train = features_train.loc[4501:5000,:]

class_train = df['class']
class_train = class_train.loc[4501:5000]
test=np.array(class_train)



Saving result_set.csv to result_set (1).csv


array([0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0.,
       4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 4., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 4., 4., 0., 0., 4., 0., 0., 4., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [239]:
val = []
for i in range(500):
  val.append(feed_forward(np.array(features_train.loc[i+4501,:])))
count=0
c = np.in1d(np.array(val),test)
for i in range(len(c)):
  if(c[i] == True):
    count +=1
print("Accuracy in %", (count/5))


Accuracy in % 77.4
